In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import folium
import geopandas as gpd
import math  

### Selection of weather stations for the Silesian Voivodeship

In [84]:
stations = pd.read_csv('data/marcin_data.csv') #SELECT name, lat,lon from bigquery-public-data.noaa_gsod.stations where  country = 'PL'
data = pd.read_csv('data/df_modified.csv')
woj = gpd.read_file('data/PRG_jednostki_administracyjne/WojewĘdztwa.shp')

In [85]:
woj_slaskie = woj[woj['JPT_KOD_JE']=='24']

In [86]:
woj_slaskie

,JPT_SJR_KO,JPT_KOD_JE,JPT_NAZWA_,JPT_ORGAN_,JPT_JOR_ID,WERSJA_OD,WERSJA_DO,WAZNY_OD,WAZNY_DO,JPT_KOD__1,...,JPT_KJ_I_1,JPT_KJ_I_2,JPT_OPIS,JPT_SPS_KO,ID_BUFOR_1,JPT_ID,JPT_KJ_I_3,Shape_Leng,Shape_Area,geometry
0,WOJ,24,ÅlÄ skie,None,0,2017-10-10,None,2012-09-26,None,None,...,24,None,None,UZG,0,1331323,None,12.136952,1.557335,"MULTIPOLYGON (((18.91685 51.09608, 18.91677 51..."


In [87]:
woj_slaskie = woj_slaskie[['JPT_KOD_JE', "geometry"]]
woj_slaskie.geometry = woj_slaskie.geometry.simplify(0.005) 
woj_geoPath = woj_slaskie.to_json()

/Users/ewa.suknarowska/opt/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[geo_column_name] = level


In [88]:
unique = stations.groupby(['name','lat','lon']).sum().reset_index()[['name','lat','lon']]

In [89]:
m = folium.Map(location=[49.793,19.057],zoom_start=7.5) 
folium.Choropleth(geo_data=woj_geoPath, fill_color='YlGn',fill_opacity=0.5,line_opacity=0.2,).add_to(m)
for lat, lon, name in zip(unique.lat, unique.lon, unique.name): 
    folium.CircleMarker([lat,lon],radius=50).add_to(m)
    folium.Marker([lat,lon], popup=name).add_to(m)

In [90]:
m

In [91]:
city = ['CIESZYN','BIELSKO-BIALA','OSWIECIM','RACIBORZ','KATOWICE','PYRZOWICE','CZESTOCHOWA','WLOSZCZOWA','STRZELCE OPOLSKIE','OLESNO','WIELUN','KRAKOW']

In [92]:
m_choosen_city = folium.Map(location=[49.793,19.057],zoom_start=7.5) 
folium.Choropleth(geo_data=woj_geoPath, fill_color='YlGn',fill_opacity=0.5,line_opacity=0.2,).add_to(m_choosen_city)
for lat, lon, name in zip(unique.lat, unique.lon, unique.name): 
    if name in city: 
        folium.CircleMarker([lat,lon],radius=120).add_to(m_choosen_city)
        folium.Marker([lat,lon], popup=name).add_to(m_choosen_city)

In [93]:
m_choosen_city

### Download data for selected weather stations

In [123]:
PROJECT = 'dwkatowice'   # CHANGE this to your GCP project
REGION  = 'region' # CHANGE this to your region
def query_to_dataframe(query):
    return pd.read_gbq(query, dialect='standard', project_id=PROJECT)

# Let's find out id of weather station in Katowice
df_city = query_to_dataframe("""
SELECT * FROM `bigquery-public-data.noaa_gsod.stations`
WHERE country = 'PL' AND (name like '%CIESZYN%' or name like '%BIELSKO-BIALA%' or name like '%OSWIECIM%' or name like '%RACIBORZ%' or name like '%KATOWICE%' or name like '%PYRZOWICE%' or name like '%CZESTOCHOWA%' or name like '%WLOSZCZOWA%' or name like '%STRZELCE OPOLSKIE%' or name like '%OLESNO%' or name like '%WIELUN%' or name like '%KRAKOW%' )
""")


Downloading: 100%|██████████| 13/13 [00:00<00:00, 13.07rows/s]


In [90]:
df_city

,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
0,125240,99999,STRZELCE OPOLSKIE,PL,None,None,50.517,18.300,+0069.0,19400201,19420630
1,125280,99999,OLESNO,PL,None,None,50.867,18.417,+0073.0,19400701,19420630
2,125550,99999,PYRZOWICE,PL,None,None,50.483,19.083,+0304.0,19881013,20021121
3,125560,99999,KRAKOW/BALICE,PL,None,EPKF,50.083,19.783,+0241.0,20020601,20160404
4,125650,99999,KRAKOW,PL,None,None,50.083,19.800,+0236.5,19520101,19600930
5,125690,99999,WLOSZCZOWA,PL,None,None,50.850,19.967,+0261.0,19400220,19420331
6,125990,99999,OSWIECIM,PL,None,None,50.033,19.217,+0073.0,19350101,19420630
7,126650,99999,CIESZYN,PL,None,None,49.750,18.633,+0312.0,19340329,20070712
8,124550,99999,WIELUN,PL,None,None,51.217,18.567,+0201.0,19391201,20200816
9,125400,99999,RACIBORZ,PL,None,None,50.050,18.200,+0206.0,19530102,20200816


In [10]:
df_city.to_csv(path_or_buf='data/df_city.csv', sep=',')

NameError: name 'df_city' is not defined

In [124]:
# And now we can build a query to get weather data
def wxquery(year):
    return """
SELECT stn,concat('{0}','-',mo,'-',da) as date, temp,  visib as visibility, wdsp wind_speed, mxpsd as max_wind_speed, gust as max_gust, max as max_temp, min as min_temp, prcp as rain_or_snow, sndp as snow, fog, rain_drizzle, snow_ice_pellets, hail, thunder 
FROM `bigquery-public-data.noaa_gsod.gsod*`
where stn in ('125240', '125280', '125550', '125560', '125650', '125690',
       '125990', '126650', '124550', '125400', '125500', '125600',
       '126000') AND _TABLE_SUFFIX = '{0}'
""".format(year)

In [125]:
weather_all = pd.DataFrame()
for year in ('2016','2017','2018','2019'):
    weather = query_to_dataframe(wxquery(year))
    weather_all = pd.concat([weather_all, weather], ignore_index=True)

Downloading: 100%|██████████| 1825/1825 [00:01<00:00, 1273.29rows/s]


In [126]:
weather_all.to_csv(path_or_buf='data/waather_all_raw.csv', sep=',')

In [108]:
weather_all.info()
#stn - id stacji pogodowej 
#date - data obcesrwacji 
#temp - średnia temperatura w fahrenheitach
#visibility - średnia widoczność w ciągu dnia w milach
#wind_speed - średnia prędkość wiatru w węzłach 
#max_wind_speed - maksymalna ciągla prędkość wiatru w węzłach 
#max_gust - maksymalny podmuch wiatru 
#max_temp - maksymalna temperatura w ciągdu dnia 
#min_temp - minimalna temperatura w ciągu dnia 
#rain_or_snow - całkowite opady deszczu lub stopiony śnieg w calach 
#snow - grubość śniegu w calch 
#fog - flaga czy była mgła 
#rain_dreizzle - mżawka 
#snow_ice_pellets - granulki lodu śnieżnego / deszcz ze śniegiem? 
#hail - grad 
#thunder - grzmot / burza 
#999,9 - brak danych 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7290 entries, 0 to 7289
Data columns (total 17 columns):
Unnamed: 0          7290 non-null int64
stn                 7290 non-null int64
date                7290 non-null object
temp                7290 non-null float64
visibility          7290 non-null float64
wind_speed          7290 non-null float64
max_wind_speed      7290 non-null float64
max_gust            7290 non-null float64
max_temp            7290 non-null float64
min_temp            7290 non-null float64
rain_or_snow        7290 non-null float64
snow                7290 non-null float64
fog                 7290 non-null int64
rain_drizzle        7290 non-null int64
snow_ice_pellets    7290 non-null int64
hail                7290 non-null int64
thunder             7290 non-null int64
dtypes: float64(9), int64(7), object(1)
memory usage: 968.3+ KB


In [130]:
weather_all = weather_all.replace({999.9: np.nan})
weather_all.rain_or_snow = weather_all.rain_or_snow.replace({99.9:np.nan})

### Converting data to the European metric system

In [131]:
weather_all['temp_c'] = weather_all.apply(lambda x: (x['temp']-32)/1.8,axis=1)
weather_all['max_temp_c'] = weather_all.apply(lambda x: (x['max_temp']-32)/1.8,axis=1)
weather_all['min_temp_c'] = weather_all.apply(lambda x: (x['min_temp']-32)/1.8,axis=1)
weather_all['delta_temp_c'] = weather_all.apply(lambda x: x['max_temp_c']-x['min_temp_c'],axis=1)
weather_all['visib_m'] = weather_all.apply(lambda x: x['visibility']/0.62137,axis=1)
weather_all['wind_speed']= pd.to_numeric(weather_all['wind_speed'], downcast="float")
weather_all['wind_speed_km_h'] = weather_all.apply(lambda x: x['wind_speed']*1.852 ,axis=1)
weather_all['max_wind_speed']= pd.to_numeric(weather_all['max_wind_speed'], downcast="float")
weather_all['max_wind_speed_km_h'] = weather_all.apply(lambda x: x['max_wind_speed']*1.852 ,axis=1)
weather_all['max_gust_km_h'] = weather_all.apply(lambda x: x['max_gust']*1.852 ,axis=1)
weather_all['rain_od_snow_m'] = weather_all.apply(lambda x: x['rain_or_snow']*0.0254 ,axis=1)
weather_all['snow_m'] = weather_all.apply(lambda x: x['snow']*0.0254 ,axis=1)

In [137]:
data[['station','date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229929 entries, 0 to 229928
Data columns (total 2 columns):
station    229929 non-null int64
date       229929 non-null object
dtypes: int64(1), object(1)
memory usage: 3.5+ MB


In [133]:
weather_all.to_csv(path_or_buf='data/weather_clean.csv', sep=',')

In [144]:
weather_all.columns

Index(['stn', 'date', 'temp', 'visibility', 'wind_speed', 'max_wind_speed',
       'max_gust', 'max_temp', 'min_temp', 'rain_or_snow', 'snow', 'fog',
       'rain_drizzle', 'snow_ice_pellets', 'hail', 'thunder', 'temp_c',
       'max_temp_c', 'min_temp_c', 'delta_temp_c', 'visib_m',
       'wind_speed_km_h', 'max_wind_speed_km_h', 'max_gust_km_h',
       'rain_od_snow_m', 'snow_m'],
      dtype='object')

### Calculation of the closest station for a given point

In [12]:
def dist(x, lon, y, lat): 
    dist=math.sqrt((x-lon)**2 + (y-lat)**2)
    return dist

In [23]:
def min_distance(x,y): 
    df_city = pd.read_csv('data/df_city.csv', sep=',')
    df_city['min'] = df_city.apply(lambda r: dist(x,r['lon'],y,r['lat']), axis = 1)
    stn = df_city [df_city['min'] == min(df_city['min'])]['usaf'].values[0]
    return stn 

In [ ]:
data['station']=data.apply(lambda x: min_distance(x['gps_x'],x['gps_y']), axis =1  )

### Combine data with weather data

In [146]:
data['date'] = data.apply(lambda x: pd.to_datetime( x['data'] ).strftime("%Y-%m-%d"), axis = 1 )
weather_all.stn = pd.to_numeric(weather_all.stn)

In [147]:
data[['station','date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229929 entries, 0 to 229928
Data columns (total 2 columns):
station    229929 non-null int64
date       229929 non-null object
dtypes: int64(1), object(1)
memory usage: 3.5+ MB


In [153]:
weather_all[['stn','date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7290 entries, 0 to 7289
Data columns (total 2 columns):
stn     7290 non-null int64
date    7290 non-null object
dtypes: int64(1), object(1)
memory usage: 114.0+ KB


In [154]:
result = pd.merge(data,weather_all , left_on=['station','date'], right_on =['stn','date'], how ='left')

In [155]:
result.to_csv(path_or_buf='data/merge_data.csv', sep=',')

### Compare weather data

In [113]:
171516/229929 #braki w danych

0.7459520112730451

In [ ]:
#stn - id stacji pogodowej 
#date - data obcesrwacji 
#temp - średnia temperatura w fahrenheitach
#visibility - średnia widoczność w ciągu dnia w milach
#wind_speed - średnia prędkość wiatru w węzłach 
#max_wind_speed - maksymalna ciągla prędkość wiatru w węzłach 
#max_gust - maksymalny podmuch wiatru 
#max_temp - maksymalna temperatura w ciągdu dnia 
#min_temp - minimalna temperatura w ciągu dnia 
#rain_or_snow - całkowite opady deszczu lub stopiony śnieg w calach 
#snow - grubość śniegu w calch 
#fog - flaga czy była mgła 
#rain_dreizzle - mżawka 
#snow_ice_pellets - granulki lodu śnieżnego / deszcz ze śniegiem? 
#hail - grad 
#thunder - grzmot / burza 
#999,9 - brak danych 

In [162]:
data.columns

Index(['Unnamed: 0', 'gps_x', 'gps_y', 'liczba_pasów_w_zdarzeniu',
       'liczba_rannych_w_zdarzeniu', 'liczba_zabitych_w_zdarzeniu',
       'liczba_kolizji', 'liczba_wypadków', 'miejscowość', 'ulica',
       'numer_domu', 'numer_drogi', 'obszar',
       'charakterystyka_miejsca_zdarzenia', 'oświetlenie',
       'warunki_atmosferyczne', 'rodzaj_zdarzenia', 'wina', 'rodzaj_drogi',
       'nawierzchnia_drogi', 'stan_nawierzchni', 'sygnalizacja_świetlna',
       'oznakowanie_poziome', 'zachowanie_kierującego', 'zachowanie_pieszego',
       'data', 'km_hm_pikietaż', 'typ_drogi', 'dobre_warunki_atmosferyczne',
       'pochmurno', 'mgła_dym', 'oślepiające_słońce', 'śniegu', 'gradu',
       'deszczu', 'silny_wiatr', 'station', 'date'],
      dtype='object')

In [164]:
weather_all.columns


Index(['stn', 'date', 'temp', 'visibility', 'wind_speed', 'max_wind_speed',
       'max_gust', 'max_temp', 'min_temp', 'rain_or_snow', 'snow', 'fog',
       'rain_drizzle', 'snow_ice_pellets', 'hail', 'thunder', 'temp_c',
       'max_temp_c', 'min_temp_c', 'delta_temp_c', 'visib_m',
       'wind_speed_km_h', 'max_wind_speed_km_h', 'max_gust_km_h',
       'rain_od_snow_m', 'snow_m'],
      dtype='object')

In [159]:
pd.set_option('display.max_columns', None)
result[['miejscowość','warunki_atmosferyczne','dobre_warunki_atmosferyczne',
       'pochmurno', 'mgła_dym', 'oślepiające_słońce', 'śniegu', 'gradu',
       'deszczu', 'silny_wiatr', 'station', 'rain_or_snow', 'snow', 'fog', 'rain_drizzle',
       'snow_ice_pellets', 'hail', 'thunder','temp_c', 'max_temp_c',
       'min_temp_c', 'delta_temp_c', 'visib_m', 'wind_speed_km_h',
       'max_wind_speed_km_h', 'max_gust_km_h', 'rain_od_snow_m', 'snow_m']]

,miejscowość,warunki_atmosferyczne,dobre_warunki_atmosferyczne,pochmurno,mgła_dym,oślepiające_słońce,śniegu,gradu,deszczu,silny_wiatr,station,rain_or_snow,snow,fog,rain_drizzle,snow_ice_pellets,hail,thunder,temp_c,max_temp_c,min_temp_c,delta_temp_c,visib_m,wind_speed_km_h,max_wind_speed_km_h,max_gust_km_h,rain_od_snow_m,snow_m
0,gliwice,dobre_warunki_atmosferyczne,1,0,0,0,0,0,0,0,125240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dąbrowa górnicza,dobre_warunki_atmosferyczne,1,0,0,0,0,0,0,0,125600,0.00,NaN,0,0,0,0,0,-8.388889,-2.000000,-12.500000,10.500000,4.023368,10.0008,18.334799,NaN,0.000000,NaN
2,tarnowskie góry,dobre_warunki_atmosferyczne,1,0,0,0,0,0,0,0,125550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tarnowskie góry,pochmurno,0,1,0,0,0,0,0,0,125550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ożarowice,dobre_warunki_atmosferyczne,1,0,0,0,0,0,0,0,125550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229924,sosnowiec,dobre_warunki_atmosferyczne,1,0,0,0,0,0,0,0,125600,0.00,NaN,0,1,0,0,0,1.500000,5.611111,-4.000000,9.611111,8.851409,13.1492,27.780000,NaN,0.000000,NaN
229925,dankowice,dobre_warunki_atmosferyczne,1,0,0,0,0,0,0,0,125990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229926,sosnowiec,dobre_warunki_atmosferyczne,1,0,0,0,0,0,0,0,125600,0.00,NaN,0,0,0,0,0,4.000000,9.000000,0.000000,9.000000,8.851409,12.9640,22.224000,NaN,0.000000,NaN
229927,zabłocie,dobre_warunki_atmosferyczne,1,0,0,0,0,0,0,0,126650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
def make_table(col_org, col_w):
    df2 = result[[col_org,col_w]].fillna('brak').groupby([col_org,col_w]).size().reset_index(name='count')
    s = result[[col_org,col_w]][col_org].value_counts()
    df2['count_proc'] = df2['count'].div(df2[col_org].map(s))
    return df2

###### mgła_dym VS fog 

In [192]:
make_table('mgła_dym','fog')

,mgła_dym,fog,count,count_proc
0,0,0,144156,0.630254
1,0,1,26588,0.116243
2,0,brak,57983,0.253503
3,1,0,146,0.121464
4,1,1,626,0.520799
5,1,brak,430,0.357737


###### śniegu/deszczu VS  / rain_dreizzle / thunder / rain_od_snow_m , snow_m 

In [209]:
result [['śniegu','deszczu','rain_drizzle', 'thunder', 'rain_od_snow_m' , 'snow_m','rain_of_snow_flag' ]]

,śniegu,deszczu,rain_drizzle,thunder,rain_od_snow_m,snow_m,rain_of_snow_flag
0,0,0,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0,0.000000,NaN,0.0
2,0,0,NaN,NaN,NaN,NaN,NaN
3,0,0,NaN,NaN,NaN,NaN,NaN
4,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
229924,0,0,1,0,0.000000,NaN,0.0
229925,0,0,NaN,NaN,NaN,NaN,NaN
229926,0,0,0,0,0.000000,NaN,0.0
229927,0,0,NaN,NaN,NaN,NaN,NaN


In [208]:
#śniegu/deszczu jest flaga wiec z rain_or_snow też zróbmy flage: 
result['rain_of_snow_flag'] = result.apply(lambda x: 1 if x['rain_od_snow_m'] > 0 else x['rain_od_snow_m'], axis =1 )
result['śniegu_lub_deszczu']=result.apply(lambda x: 1 if x['śniegu']+x['deszczu'] > 0 else 0, axis =1 )

In [212]:
make_table('śniegu_lub_deszczu','rain_of_snow_flag')

,śniegu_lub_deszczu,rain_of_snow_flag,count,count_proc
0,0,0,83325,0.430887
1,0,1,61034,0.315617
2,0,brak,49021,0.253496
3,1,0,5350,0.146379
4,1,1,21807,0.596651
5,1,brak,9392,0.256970


In [213]:
result['snow_flag'] = result.apply(lambda x: 1 if x['snow_m'] > 0 else x['snow_m'], axis =1 )

In [214]:
make_table('śniegu','snow_flag')

,śniegu,snow_flag,count,count_proc
0,0,1,15537,0.069816
1,0,brak,207006,0.930184
2,1,1,2779,0.376252
3,1,brak,4607,0.623748


In [215]:
#TODO: rain_dreizzle / thunder

###### gradu  VS snow_ice_pellets / hail 

In [218]:
result[['gradu','snow_ice_pellets','hail']]

,gradu,snow_ice_pellets,hail
0,0,NaN,NaN
1,0,0,0
2,0,NaN,NaN
3,0,NaN,NaN
4,0,NaN,NaN
...,...,...,...
229924,0,0,0
229925,0,NaN,NaN
229926,0,0,0
229927,0,NaN,NaN


In [219]:
make_table('gradu','snow_ice_pellets')

,gradu,snow_ice_pellets,count,count_proc
0,0,0,138201,0.621008
1,0,1,28016,0.125890
2,0,brak,56326,0.253102
3,1,0,118,0.015976
4,1,1,5181,0.701462
5,1,brak,2087,0.282562


In [220]:
make_table('gradu','hail')

,gradu,hail,count,count_proc
0,0,0,164605,0.739655
1,0,1,1612,0.007244
2,0,brak,56326,0.253102
3,1,0,5003,0.677363
4,1,1,296,0.040076
5,1,brak,2087,0.282562


###### silny_wiatr VS wind_speed_km_h/ max_wind_speed_km_h / max_gust_km_h

In [224]:
result[['silny_wiatr','wind_speed_km_h','max_wind_speed_km_h','max_gust_km_h']].groupby('silny_wiatr').agg(['min', 'max'])

wind_speed_km_h              max_wind_speed_km_h               \
                        min          max                 min          max   
silny_wiatr                                                                 
0                    2.4076  1851.814845              3.5188  1851.814845   
1                    4.2596    53.522799              7.2228    72.042803   

            max_gust_km_h           
                      min      max  
silny_wiatr                         
0                 21.6684  122.232  
1                 25.1872  122.232

In [225]:
#TODO: wind 
#TODO: czy da się sprognozować dobre waruni pogodowe na podstawie dodatkowych danych? 